# Práctica de despolarización lidar

Éste es el código que se utilizará para el desarrollo de la práctica de lidar elástico. Para ello el/la alumno/a solo tendrá que modificar las sentencias que se encuentran entre:

#------------------------------------------------------------------------------------------------------------------------------

         Modifique SOLO aquí dentro
#------------------------------------------------------------------------------------------------------------------------------

Importante:
- Un error recurrente puede ser que las rutas se especifiquen con '\\' como separador entre carpetas, mientras que el lenguaje Python utiliza '\\\\' (doble). 
- También se recomienda correr una a una las celdas, y no correr de nuevo una celda mientras se está ejecutando (el software puede quedarse colgado). Si mientras una celda se está ejecutando, se realiza algún cambio en el código de esa misma celda y queremos volver a ejecutarla, deberemos esperar a que termine o reiniciar los kernels.
- Si lo desea, puede jugar con el resto del código.
- Para el desarrollo de la práctica y elaboración del informe se pueden emplear resultados obtenidos en otras prácticas, que sean relativos a la misma situación atmosférica.

**IMPORTACIÓN DE LIBRERÍAS Y ESPECIFICACIÓN DE RUTAS**

En primer lugar **importamos las librerias** necesarias. En el caso de que no se encuentre alguno de los módulos al correr esta celda tendremos que instalar dicho módulo en Anaconda Navigator -> Environments o a traves de Anaconda Prompt.

In [ ]:
import os
from pathlib import Path

import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter

#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import date_quicklook
from gfatpy.lidar.lidar_utils import signal_to_rcs
from gfatpy.utils.plot import font_axes

%matplotlib inline 


También, antes de comenzar con el tratamiento de los ficheros, definimos la ruta en la que se encuentran los datos.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
# databasepath = Path(r'G:\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_estructuras\data\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
databasepath = Path(r'C:\Users\Usuario\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_despolarizacion\data\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
outputpath =   Path(r'C:\Users\Usuario\Downloads\geomet_output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

databasepath.is_dir(), outputpath.is_dir() #Comprueba que ambas carpetas existen.

Por otra parte, especificaremos en este momento **la fecha** con la que vamos a trabajar. Esto evitará posibles problemas más adelante y también nos permitirá leer todos los ficheros para ese día concreto. El formato será 'YYYYMMDD', con Y el año, M el mes y D el día. Es necesario tener en cuenta que los datos para ese día tienen que estár guardados en una carpeta cuyo nombre sea la fecha con el formato anteriormente especificado, y que esta carpeta se encuentre dentro de database_path. A diferencia de otras prácticas, en esta no será necesario indicar explicitamente la ruta exacta de cada uno de los ficheros.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
filepath = databasepath / '2019' / '08' / '08' / 'mhc_1a_Prs_rs_xf_20190808.nc'   #Lidar product path. Example : 'C:\\Users\\user\\Desktop\\data\\1a\\2019\\01\\01\\mhc_1a_Prs_rs_xf_20190101.nc' 
dcfilepath = databasepath / '2019' / '08' / '08' / 'mhc_1a_Pdc_rs_xf_20190808_0628.nc' #Lidar dark current product path
ini_date = '20190808T180000'   #Initial date and time. Format 'yyyymmddThhmmss', where T represents literally the letter T. Example: '20190101T200000'
end_date = '20190808T183000'   #Same format than ini_date
#------------------------------------------------------------------------------------------------------------------------------

if os.path.exists(filepath) and os.path.exists(dcfilepath):
    channels = ["532xta"]
    data = preprocess(filepath, dc_fl=dcfilepath, channels=channels, ini_date=ini_date, end_date=end_date)
else:
    print("One (or both) of the given files can not be found.")

In [ ]:
fig,ax = plt.subplots()
signal_to_rcs(data['signal_532xta'], data.range).plot(y= 'range',ax=ax, vmin=0,vmax=5e6,cmap='jet')
ax.set_ylim(0,10000.)

mask = signal_to_rcs(data['signal_532xta'], data.range) > 1e6

fig,ax = plt.subplots()
data['lvd_532xa'].where(mask).plot(y= 'range',ax=ax, vmin=0,vmax=0.5,cmap='jet')
ax.set_ylim(0,10000.)



**REPRESENTACIÓN DE SEÑAL CORREGIDA DE RANGO Y DESPOLARIZACIÓN VOLÚMICA LINEAL**

En cualquier análisis de medidas lidar, el primer paso es **representar los perfiles de señal corregida de rango** y en nuestro caso también **los perfiles de despolarización volúmica lineal** obtenidos con el instrumento, para contextualizar la situación atmosférica y estimar el valor de altura de referencia que se tendrán que emplear en cálculos posteriores.

En la celda de código siguiente representaremos los diferentes perfiles medidos para un día concreto, en forma de los comúnmente conocidos quicklooks, de la señal corregida de rango. Debemos tener en cuenta los siguientes parámetros de entrada:
- y_min, y_max: son las alturas mínima y máxima, respectivamente, dadas en metros asl para representar el perfil de señal corregida de rango.
- ini_time, end_time: son los valores mínimo y máximo, respectivamente, del eje de tiempos. Es necesario utilizar el formato 'YYYYMMDDTHH:MM:SS', donde Y se corresponde con el año, M con el mes, D con el día, H con la hora, M con los minutos, S con los segundos (que debe ser un entero), y T es literalmente la letra T. Como anteriormente ya se ha especificado la fecha, puede espeficicarse solamente 'HH:MM:SS' (ini_time = date + 'HH:MM:SS'). Si se desea utilizar estos parámetros de entrada deben indicarse ambos, no exclusivamente uno de ellos. De no querer utilizarlos, escriba None.
- c_min, c_max: son los valores mínimo y máximo, respectivamente, del eje de color. En este caso representamos la RCS y sus unidades son arbitrarias.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
qlpath = outputpath / 'quicklooks' #Output quicklook folder path. It is suggested to mantain the proposed path
ini_date = '20190808' #Data for which quicklooks will be obtained. Format yyyymmdd
end_date = ini_date
#------------------------------------------------------------------------------------------------------------------------------
channels = ["lvd_532xa"]
date_quicklook(
    ini_date,
    channels2plot=channels,
    path1a=databasepath,
    figpath=qlpath,
    colorbar_range=(0,0.5),
)


En la celda de código siguiente representaremos los diferentes perfiles medidos para un día concreto, en forma de los comúnmente conocidos quicklooks, de la despolarización volúmica lineal. Debemos tener en cuenta los siguientes parámetros de entrada:
- y_min, y_max.
- ini_time, end_time.
- c_min, c_max.

In [ ]:
data

In [ ]:
channels = ["lvd532xa"]
date_quicklook(
    ini_date,
    channels2plot=channels,
    path1a=databasepath,
    figpath=qlpath,
    colorbar_range=(0,0.5),
)


**PERFILES PROMEDIO**

Con la siguiente celda se permitirá **obtener los perfiles promedios** necesarios, para los momentos del día y variables anteriores (señal corregida de rango y despolarización volúmica lineal). Serán necesarios:
- y_min, y_max.
- ini_time, end_time: son los valores mínimo y máximo, respectivamente, del eje de tiempos entre los que vamos a promediar. Es necesario utilizar el formato 'YYYYMMDDTHH:MM:SS', anteriormente explicado. En este caso no se permite no dar ninguno de estos parámetros.
- RCS_min, RCS_max: son los valores mínimo y máximo, respectivamente, del perfil promediado de la RCS.
- d_min, d_max: son los valores mínimo y máximo, respectivamente, del perfil promediado de la despolarización volúmica lineal.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------ 
ini_time = date + '' #Sets the minimum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'
end_time = date + '' #Sets the maximum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'
y_min =    #Sets the bottom value of the vertical axis (altitude) in meters asl
y_max =    #Sets the top value of the vertical axis (altitude) in meters asl
RCS_min =  #Sets the minimum value of the RCS plot
RCS_max =  #Sets the maximum value of the RCS plot
d_min =    #Sets the minimum value of the depolarization plot
d_max =    #Sets the maximum value of the depolarization plot
#------------------------------------------------------------------------------------------------------------------------------
average_profiles(files_list, ini_time, end_time, y_min, y_max, RCS_min, RCS_max, d_min, d_max)

**REPRESENTACIÓN DEL COEFICIENTE DE RETRODISPERSIÓN DE PARTÍCULAS Y RAZÓN DE RETRODISPERSIÓN**

A continuación, y con objeto de finalmente obtener el perfil de la razón de despolarización de partículas, es necesario estudiar la retrodispersión de la situación atmosférica. Por ello, **representaremos los perfiles del coeficiente de retrosipersión de partículas y razón de retrosipersión**. 

En la celda siguiente se representarán los perfiles del coeficiente de retrodispersión de partículas a traves de un quicklook. Debemos tener en cuenta los siguientes parámetros de entrada:
- h_ref, lr, klett_window: son la altura de referencia (metros asl), razón lidar (sr) y ancho de la ventana (metros) necesarios para aplicar el algoritmo de inersión de Klett y obtener los perfiles de rerodispersión. 
- y_min, y_max.
- ini_time, end_time: serán los valores mínimo y máximo, respectivamente, del eje de tiempos. Es necesario utilizar el formato 'YYYYMMDDTHH:MM:SS', anteriormente explicado. Si se desea utilizar estos parámetros de entrada deben indicarse ambos, no exclusivamente uno de ellos. De no querer utilizarlos, escriba a None.
- c_min, c_max: serán los valores mínimo y máximo, respectivamente, del eje de color. En este caso se representa coeficiente de retrodispersión de partículas y será necesario indicar estos valores en Mm$^{-1}$·sr$^{-1}$ (ó 10$^{-6}$m$^{-1}$·sr$^{-1}$)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
h_ref =         #Sets the reference altitude to perform the Klett inversion
lr =            #Sets the lidar ratio to perform the Klett inversion
klett_window =  #Sets the vertical window to perfor the Klett inversion
y_min =         #Sets the bottom value of the vertical axis (altitude) in meters asl
y_max =         #Sets the top value of the vertical axix (altitude) in meters asl
ini_time = date + '' #Sets the minimum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'. If you prefer not to set one write ini_time = None
end_time = date + '' #Sets the maximum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'. If you prefer not to set one write end_time = None
c_min = 0       #Sets the minimum value of the color axis.
c_max = 3.5     #Sets the maximum value of the color axis. As a first approach it is recommended to use the default value
#------------------------------------------------------------------------------------------------------------------------------
plot_backs_quicklook(files_list, h_ref, lr, klett_window, y_min, y_max, ini_time, end_time, c_min, c_max)

En la celda siguiente se representarán los perfiles de razón de retrodispersión a través de un quicklook. Debemos tener en cuenta los siguientes parámetros de entrada:
- h_ref, lr, klett_window.
- y_min, y_max.
- ini_time, end_time.
- c_min, c_max.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
h_ref =         #Sets the reference altitude to perform the Klett inversion
lr =            #Sets the lidar ratio to perform the Klett inversion
klett_window =  #Sets the vertical window to perfor the Klett inversion
y_min =         #Sets the bottom value of the vertical axis (altitude) in meters asl
y_max =         #Sets the top value of the vertical axix (altitude) in meters asl
ini_time = date + '' #Sets the minimum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'. If you prefer not to set one write ini_time = None
end_time = date + '' #Sets the maximum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'. If you prefer not to set one write end_time = None
c_min = 0       #Sets the minimum value of the color axis.
c_max = 6       #Sets the maximum value of the color axis. As a first approach it is recommended to use the default value
#------------------------------------------------------------------------------------------------------------------------------
plot_backsR_quicklook(files_list, h_ref, lr, klett_window, y_min, y_max, ini_time, end_time, c_min, c_max)

**CÁLCULO DE LA RAZÓN DE DESPOLARIZACIÓN LINEAL DE PARTÍCULAS**

Finalmente, y como se ha detallado en la teoría, ya disponemos de los diferentes parámetros necesarios para calcular la despolarización lineal de partículas: la despolarización volúmica lineal, la razón de despolarización y el coeficiente de retrodispersión de partículas (la despolarización molecular se aproximará por 0.014).

En la celda siguiente se representarán los perfiles de la razón de despolarización de partículas a traves de un quicklook. Debemos tener en cuenta los siguientes parámetros de entrada:
- h_ref, lr, klett_window. 
- y_min, y_max.
- R_limit: será el valor minimo de razón de retrodispersión para el que se calculará la despolarización lineal de partículas.
- ini_time, end_time: serán los valores mínimo y máximo, respectivamente, del eje de tiempos. Es necesario utilizar el formato 'YYYYMMDDTHH:MM:SS', anteriormente explicado. Si se desea utilizar estos parámetros de entrada deben indicarse ambos, no exclusivamente uno de ellos. De no querer utilizarlos, igualadlos a None.
- c_min, c_max.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
h_ref =         #Sets the reference altitude to perform the Klett inversion
lr =            #Sets the lidar ratio to perform the Klett inversion
klett_window =  #Sets the vertical window to perfor the Klett inversion
R_limit =       #Sets the minimum value to calculare particle depolarization ratio
y_min =         #Sets the bottom value of the vertical axis (altitude) in meters asl
y_max =         #Sets the top value of the vertical axix (altitude) in meters asl
ini_time = date + '' #Sets the minimum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'. If you prefer not to set one write ini_time = None
end_time = date + '' #Sets the maximum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'. If you prefer not to set one write end_time = None
c_min =         #Sets the minimum value of the color axis.
c_max =         #Sets the maximum value of the color axis. As a first approach it is recommended to use the default value
#------------------------------------------------------------------------------------------------------------------------------
plot_partdepol_quicklook(files_list, h_ref, lr, klett_window, R_limit, y_min, y_max, ini_time, end_time, c_min, c_max)

**REPRESENTACIÓN DE LAS VARIABLES MÁS SIGNIFICATIVAS**

Finalmente se representarán conjuntamente, para un intervalo temporal, los perfiles promediados de las variables con las que más fácilmente se puede interpretar la situación atmosférica de estudio: coeficiente de retrodispersión de partículas, razón de retrodispersión y razones de despolarización volumica y de partículas. Para ello será necesario indicar los siguientes parámetros de entrada:
- h_ref, lr, klett_window. 
- R_limit: será el valor minimo de razón de retrodispersión para el que se calculará la despolarización lineal de partículas.
- y_min, y_max.
- ini_time, end_time.
- beta_min, beta_max: serán los valores mínimo y máximo, respectivamente, de la gráfica del coeficiente de retrodispersión de partículas.
- R_min, R_max: serán los valores mínimo y máximo, respectivamente, de la gráfica de la razón de retrodispersión.
- d_min, d_max: serán los valores mínimo y máximo, respectivamente, de la gráfica de las razones de despolarización.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
h_ref =         #Sets the reference altitude to perform the Klett inversion
lr =            #Sets the lidar ratio to perform the Klett inversion
klett_window =  #Sets the vertical window to perfor the Klett inversion
R_limit =       #Sets the minimum value to calculare particle depolarization ratio
y_min =         #Sets the bottom value of the vertical axis (altitude) in meters asl
y_max =         #Sets the top value of the vertical axix (altitude) in meters asl
ini_time = date + '' #Sets the minimum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'
end_time = date + '' #Sets the maximum value of the time axis, with format 'YYYYMMDDTHH:MM:SS'
beta_min =      #Sets the minimum value of the backscatter plot
beta_max =      #Sets the maximum value of the backscatter plot
R_min =         #Sets the minimum value of the backscatter ratio plot
R_max =         #Sets the maximum value of the backscatter ratio plot
d_min =         #Sets the minimum value of the depolarization ratio plot
d_max =         #Sets the maximum value of the depolarization ratio plot
#------------------------------------------------------------------------------------------------------------------------------
plot_relevant_variables(files_list, ini_time, end_time, h_ref, lr, klett_window, R_limit, y_min, y_max, beta_min, beta_max,
                        R_min, R_max, d_min, d_max)